# TREBL Full Analysis Example

This notebook demonstrates a comprehensive TREBL analysis workflow with:
- **Error correction enabled** (improves accuracy by salvaging sequences)
- **Both simple and directional/complex UMI deduplication** (comprehensive PCR artifact removal)

This approach is recommended for final, publication-quality analysis when accuracy is the priority.

**Note:** Error correction and complex UMI deduplication significantly increase processing time. For large datasets, consider submitting as a Savio job (see `examples/savio_jobs/full_analysis_job.sh`).

## Setup and Imports

In [ ]:
import sys
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
from tqdm import tqdm

from trebl_tools import (
    initial_map,
    map_refiner,
    complexity,
    finder,
    preprocess,
    error_correct,
    plotting,
    umi_deduplicate,
    pipelines
)

## Initialize Pipeline

Key settings for full analysis:
- `error_correction=True` - Enables UMI-tools based error correction
- `test_n_reads` - Optional: Set to a number for testing with subset of data

In [ ]:
# Initialize pipeline with error correction
pipeline = pipelines.TreblPipeline(
    db_path="full_analysis.db",
    design_file_path="path/to/your/design_file.txt",  # Update this path
    error_correction=True,  # Enable error correction for full analysis
    output_path="output/full_analysis"
    # test_n_reads=100000  # Uncomment to test with first 100k reads
)

## Step 1: TREBL Mapping with Error Correction

Define barcodes and run initial mapping with error correction enabled.

In [ ]:
# Define barcodes to search for in reads
AD = finder.Barcode(
    name="AD",
    preceder="GGCTAGC",
    post="TGACTAG",
    length=120
)

AD_BC = finder.Barcode(
    name="AD_BC",
    preceder="CGCGCC",
    post="GGGCCC",
    length=11
)

RT_BC = finder.Barcode(
    name="RT_BC",
    preceder="CTCGAG",
    post="GGCCGC",
    length=14
)

# Combine barcodes
bc_objects = [AD, AD_BC, RT_BC]

In [ ]:
# Specify sequencing file(s)
step1_seq_file = "path/to/your/step1_sequencing_file.fastq"  # Update this path
# Can be a single file (string) or multiple files (list of strings)
# Supported formats: .fastq or .fastq.gz

In [ ]:
# Plot reads distribution
# NOTE: For large files (>10M reads), consider submitting this as a Savio job
# See examples/savio_jobs/full_analysis_job.sh for job submission example

pipeline.step1_reads_distribution(
    seq_file=step1_seq_file,
    bc_objects=bc_objects,
    reverse_complement=True
)
# Produces histogram of reads per barcode
# Helps pick appropriate reads_threshold for filtering

In [ ]:
# Run Step 1 mapping with error correction
# Error correction happens automatically based on pipeline initialization
step1_map = pipeline.run_step_1(
    seq_file=step1_seq_file,
    bc_objects=bc_objects,
    column_pairs=[("RT_BC", "AD")],  # Check for collisions between RT_BC and AD
    reads_threshold=10,  # Minimum reads to keep a barcode
    reverse_complement=False
)
# Returns DataFrame of Step 1 mapping
# Error correction will salvage sequences similar to high-read sequences
# Saves CSV, loss table visualization, and optional loss table CSV

## Step 2: TREBL Step 2 Mapping

Step 2 processes AD and RT libraries that are now in separate sequencing files.

**Note:** Step 1 must be completed successfully before running Step 2.

In [ ]:
# Sequencing files for AD and RT (Step 2)
step2_AD_seq_file = "path/to/your/step2_AD_file.fastq"  # Update this path
step2_RT_seq_file = "path/to/your/step2_RT_file.fastq"  # Update this path
# Can be single files or lists of files; .fastq or .fastq.gz

In [ ]:
# Plot Step 2 reads distribution
# NOTE: For large files (>10M reads), consider submitting this as a Savio job

pipeline.step2_reads_distribution(
    AD_seq_file=step2_AD_seq_file,
    AD_bc_objects=AD_bc_objects,
    RT_seq_file=step2_RT_seq_file,
    RT_bc_objects=RT_bc_objects,
    reverse_complement=True
)
# Produces histograms for AD and RT reads
# Helps pick reads_threshold_AD and reads_threshold_RT

In [ ]:
# Run Step 2 mapping with error correction
# Error correction happens automatically based on pipeline initialization
step2 = pipeline.run_step_2(
    AD_seq_file=step2_AD_seq_file,
    AD_bc_objects=AD_bc_objects,
    RT_seq_file=step2_RT_seq_file,
    RT_bc_objects=RT_bc_objects,
    reverse_complement=True,
    reads_threshold_AD=10,
    reads_threshold_RT=10,
    step1_map_csv_path="output/full_analysis/step1_AD_AD_BC_RT_BC_error_corrected_designed.csv"  # Update with your step1 CSV path
)

# Extract outputs
AD_step2 = step2["AD_step2"]
RT_step2 = step2["RT_step2"]
step1_overlap = step2["step1_overlap"]

# The overlap shows how well Step 2 data aligns with Step 1 mapping

## TREBL Experiment with Both UMI Deduplication Methods

Process the full TREBL experiment using both simple and directional/complex UMI deduplication.

**UMI Deduplication Methods:**
- **Simple:** Counts unique UMI sequences (faster, good baseline)
- **Directional/Complex:** Uses UMI-tools directional algorithm to account for PCR and sequencing errors in UMIs (more accurate)

In [ ]:
# Define UMI objects
AD_UMI = finder.Barcode(
    name="UMI",
    preceder="TGATTT",
    post="",
    length=12
)

RT_UMI = finder.Barcode(
    name="UMI",
    preceder="TGTCAC",
    post="",
    length=12
)

# Separate barcode objects
AD_bc_objects = [AD, AD_BC]  # AD and AD_BC barcodes
RT_bc_objects = [RT_BC]      # Reporter barcodes

In [ ]:
# Collect sequencing files
trebl_AD_seq_files = glob.glob("path/to/AD_assembled/*")  # Update this path
trebl_RT_seq_files = glob.glob("path/to/RT_assembled/*")  # Update this path

In [ ]:
# Plot reads distribution for all files
# NOTE: For large files, consider submitting this as a Savio job
# See examples/savio_jobs/full_analysis_job.sh for job submission example

pipeline.trebl_experiment_reads_distribution(
    AD_seq_files=trebl_AD_seq_files,
    AD_bc_objects=AD_bc_objects,
    RT_seq_files=trebl_RT_seq_files,
    RT_bc_objects=RT_bc_objects,
    reverse_complement=True
)
# Generates histograms for all AD and RT files

In [ ]:
# Run TREBL experiment with BOTH simple and directional/complex UMI deduplication
trebl_results = pipeline.trebl_experiment_analysis(
    AD_seq_files=trebl_AD_seq_files,
    AD_bc_objects=AD_bc_objects,
    RT_seq_files=trebl_RT_seq_files,
    RT_bc_objects=RT_bc_objects,
    reverse_complement=True,
    step1_map_csv_path="output/full_analysis/step1_AD_AD_BC_RT_BC_error_corrected_designed.csv",  # Update with your step1 CSV path
    AD_umi_object=AD_UMI,
    RT_umi_object=RT_UMI,
    umi_deduplication='both'  # Use BOTH simple and directional/complex deduplication
)

# Access results
# Results contain merged counts from both deduplication methods
AD_results = trebl_results["AD_results"]
RT_results = trebl_results["RT_results"]

## Understanding the Results

When `umi_deduplication='both'` is used:
- Results include columns for both simple UMI counts and directional/complex UMI counts
- The directional method typically gives lower counts as it removes UMIs that likely resulted from PCR/sequencing errors
- For most analyses, the directional/complex counts are recommended for final results

In [ ]:
# Example: Compare simple vs complex deduplication results
print("AD Results columns:", AD_results.columns.tolist())
print("\nFirst few rows of AD results:")
AD_results.head()

## Next Steps

After completing this full analysis:

1. **Review outputs** in the `output/full_analysis` directory
2. **Check loss tables** - You should see an additional 'error_corrected' step
3. **Compare UMI deduplication methods** in the output CSVs
4. **Calculate activity scores** using the results (see documentation)

### Key Differences from Quick Start:

- **Error correction** salvages more reads by correcting sequences similar to high-confidence barcodes
- **Directional UMI deduplication** provides more accurate counts by handling UMI errors
- **Processing time** is significantly longer but provides higher quality results

### Cleanup

After analysis is complete, you can delete the DuckDB database:

In [ ]:
# import os
# os.remove("full_analysis.db")